In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

In [3]:
work_dir = '/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/'
train_dir = os.path.join(work_dir, 'Data/Train_test_val/Train/')
test_dir = os.path.join(work_dir, 'Data/Train_test_val/Test/')
val_dir = os.path.join(work_dir, 'Data/Train_test_val/Val/')

easyocr_dir = os.path.join(work_dir, 'Prediction/EASYOCR/image/')
easyocr_non_bgr_dir = os.path.join(work_dir, 'Prediction/EASYOCR/non_background/')
easyocr_non_bgr_pre_dir = os.path.join(work_dir, 'Prediction/EASYOCR/points_text_non_background/')
easyocr_pre_dir = os.path.join(work_dir, 'Prediction/EASYOCR/points_text/')
model_dir = os.path.join(work_dir, 'Model/')

##install

In [4]:
import ast
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
import os
from shapely.geometry import Polygon
from statistics import mean
from os import system

In [5]:
!pip install jiwer
from jiwer import wer, cer

In [6]:
!pip install easyocr

In [7]:
!pip install git+git://github.com/jaidedai/easyocr.git

  Cloning git://github.com/jaidedai/easyocr.git to /tmp/pip-req-build-xa5nj3p0
  Running command git clone -q git://github.com/jaidedai/easyocr.git /tmp/pip-req-build-xa5nj3p0


In [8]:
#!pip uninstall opencv-python-headless 
!pip install opencv-python-headless==4.1.2.30

In [9]:
import easyocr

##PREPROCESSING

###SEGMENTATION

####install

In [10]:
!cp /content/detectron2/detectron2/engine/defaults.py /content

cp: cannot stat '/content/detectron2/detectron2/engine/defaults.py': No such file or directory


In [11]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

1.10.0+cu111 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [12]:
!python3 -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-p8qcveqx
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-p8qcveqx


In [13]:
'''efficientnet_pytorch==0.7.0
albumentations==0.5.2
vietocr==0.3.5
transformers==3.4.0
vncorenlp==1.0.3
scikit-learn==0.22.2.post1
torch==1.7.1
torchvision==0.8.2
imutils==0.5.3'''

'efficientnet_pytorch==0.7.0\nalbumentations==0.5.2\nvietocr==0.3.5\ntransformers==3.4.0\nvncorenlp==1.0.3\nscikit-learn==0.22.2.post1\ntorch==1.7.1\ntorchvision==0.8.2\nimutils==0.5.3'

In [14]:
!python -m pip install detectron2==0.6 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch==1.7.1/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch==1.7.1/index.html


In [15]:
!pip install pyyaml==5.1

In [16]:
#!python3 -m pip install 'git+https://github.com/hoainv99/mc-ocr.git'

In [17]:
!git clone -q https://github.com/hoainv99/mc-ocr.git

fatal: destination path 'mc-ocr' already exists and is not an empty directory.


####function

In [18]:
!pip install pyyaml==5.1
try:
  from detectron2.engine import DefaultPredictor
except: 
  !pip install pyyaml==5.1
from detectron2.config import get_cfg
import os
import cv2
import numpy as np
from detectron2 import model_zoo
from google.colab.patches import cv2_imshow

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Model/Segmentation_Image/model_segmentation.pth"
#cfg.MODEL.WEIGHTS = 'model_segmentation.pth'  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.95   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


In [19]:
def get_segment_crop(img, tol=0, mask=None):
    if mask is None:
        mask = img > tol
    return img[np.ix_(mask.any(1), mask.any(0))]

def segment_single_images(image, save_img=False):
    error_ims = []
    segmen_info = []

#   image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    output_predictor = predictor(image)
    if output_predictor['instances'].pred_masks.shape[0] > 1:
        mask_check = output_predictor['instances'].pred_masks.cpu().numpy()
        masks = output_predictor['instances'].pred_masks.cpu().numpy()
        mask_binary = masks[np.argmax(np.sum(masks, axis=(1, 2))) ,:,:]

    else:
        mask_binary = np.squeeze(output_predictor['instances'].pred_masks.permute(1, 2, 0).cpu().numpy())

    try:
        crop_mask = get_segment_crop(img = image, mask = mask_binary)
        
    except ValueError:
        print("error")
    origin_mask = cv2.cvtColor(np.float32(mask_binary) * 255.0, cv2.COLOR_GRAY2RGB)

    for j in range(image.shape[2]):
        image[:,:,j] = image[:,:,j] * origin_mask[:,:,j] * 255

    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

####Drop background

In [20]:
test_dir

'/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Data/Train_test_val/Test/'

In [21]:
data_path = test_dir
saving_path = easyocr_non_bgr_dir

In [22]:
lst_file =[]
for filename in os.listdir(data_path):
  lst_file.append(filename)

In [23]:
len(lst_file)
#lst_file.pop(-1)

90

In [24]:
#sorted(lst_file)

In [25]:
'''
lst_file.pop(-1)
lst_file.pop(-1)'''

'\nlst_file.pop(-1)\nlst_file.pop(-1)'

In [26]:
def save_segmentation_image(saving_path,data_path, lst_file):
  leng = len(lst_file)
  for i in tqdm (range(leng), desc="Loading…", ascii=False, ncols=100):
    #print(i)
    try:
      image = cv2.imread(data_path + lst_file[i])
      after = segment_single_images(image, False)
      cv2.imwrite(saving_path+lst_file[i], after)
      #print(i)
    except:
      #print("error>>",i)
      cv2.imwrite(saving_path+lst_file[i], after)

In [27]:
save_segmentation_image(saving_path,data_path, lst_file)

Loading…:   0%|                                                              | 0/90 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Loading…:  49%|█████████████████████████▉                           | 44/90 [00:26<00:29,  1.58it/s]

error


Loading…:  78%|█████████████████████████████████████████▏           | 70/90 [00:41<00:09,  2.10it/s]

error


Loading…:  80%|██████████████████████████████████████████▍          | 72/90 [00:42<00:08,  2.12it/s]

error


Loading…:  90%|███████████████████████████████████████████████▋     | 81/90 [00:46<00:04,  2.07it/s]

error


Loading…: 100%|█████████████████████████████████████████████████████| 90/90 [00:50<00:00,  1.77it/s]


####Contour

In [28]:
print(cv2.__version__)

4.1.2


In [29]:
'''
import cv2

# Load image, convert to grayscale, and find edges
image = cv2.imread('/content/test_predict/after.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]

# Find contour and sort by contour area
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

# Find bounding box and extract ROI
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    ROI = image[y:y+h, x:x+w]
    break

cv2_imshow(ROI)
cv2.imwrite('ROI.png',ROI)
cv2.waitKey()
'''

"\nimport cv2\n\n# Load image, convert to grayscale, and find edges\nimage = cv2.imread('/content/test_predict/after.jpg')\ngray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\nthresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]\n\n# Find contour and sort by contour area\ncnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\ncnts = cnts[0] if len(cnts) == 2 else cnts[1]\ncnts = sorted(cnts, key=cv2.contourArea, reverse=True)\n\n# Find bounding box and extract ROI\nfor c in cnts:\n    x,y,w,h = cv2.boundingRect(c)\n    ROI = image[y:y+h, x:x+w]\n    break\n\ncv2_imshow(ROI)\ncv2.imwrite('ROI.png',ROI)\ncv2.waitKey()\n"

##EASY OCR MODEL


In [30]:
reader = easyocr.Reader(['vi']) # this needs to run only once to load the model into memory
#result = reader.readtext('test.jpg')

In [31]:
import cv2
from google.colab.patches import cv2_imshow

In [32]:
from google.colab import files

In [33]:
def SingleImageLabel(result, file_name):
  label = []
  for i in range(len(result)):
    temp_dct = {}
    temp_dct.update({"transcription":result[i][1]})
    temp_dct.update({"points":result[i][0]})
    temp_dct.update({"difficult":False})
    label.append(temp_dct)
  return label

def UpdateLabel(label, filename):
  label_file = label_file + model + str("/")+str(filename)+str("\t")+str(label)+str("\n")
  return label_file

def SaveLabel(path, label_file):
  #print(label_file)
  f = open(path, "w")
  f.write(label_file)
  f.close()

#get all file names
#import os
model = "EASYOCR"
#--------------------------------path = "/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/Test/"+model+"/"
lst_file =[]
#for filename in os.listdir(path):
  #lst_file.append(filename)

def save_segmentation_image(path, lst_file):
  leng = len(lst_file)
  for i in range(leng):
    print(i)
    image = cv2.imread(path + lst_file[i])
    try:
      after = segment_single_images(image, False)
      cv2.imwrite(path+lst_file[i], after)
      print(i)
    except: 
      print("error>>",i)
      
def handle_bb(path, model, lst_file, label_file):
  for i in range(len(lst_file)):
    result = reader.readtext(path + lst_file[i])

    label = SingleImageLabel(result, lst_file[i])
    #print("label  ", label)

    #updatelabel-----
    print("UpdateLabel")
    label_file = label_file+ str("/")+str(lst_file[i])+str("\t")+str(label)+str("\n")

  SaveLabel(path+"Label.txt", label_file)
  print("Done")

In [34]:

#label_file = ""
#handle_bb(path, "EASYOCR", lst_file, label_file)

In [35]:
#replace to putText
'''
from PIL import Image, ImageDraw, ImageFont
from typing import Tuple
def cv2_img_add_text(img, text, left_corner: Tuple[int, int],
                     text_rgb_color=(255, 0, 0), text_size=24, font='arial.ttc', **option):
    pil_img = img
    if isinstance(pil_img, np.ndarray):
        pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)
    font_text = ImageFont.truetype(font=font, size=text_size, encoding=option.get('encoding', 'utf-8'))
    draw.text(left_corner, text, text_rgb_color, font=font_text)
    cv2_img = cv2.cvtColor(np.asarray(pil_img), cv2.COLOR_RGB2BGR)
    if option.get('replace'):
        img[:] = cv2_img[:]
        return None
    return cv2_img'''

"\nfrom PIL import Image, ImageDraw, ImageFont\nfrom typing import Tuple\ndef cv2_img_add_text(img, text, left_corner: Tuple[int, int],\n                     text_rgb_color=(255, 0, 0), text_size=24, font='arial.ttc', **option):\n    pil_img = img\n    if isinstance(pil_img, np.ndarray):\n        pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))\n    draw = ImageDraw.Draw(pil_img)\n    font_text = ImageFont.truetype(font=font, size=text_size, encoding=option.get('encoding', 'utf-8'))\n    draw.text(left_corner, text, text_rgb_color, font=font_text)\n    cv2_img = cv2.cvtColor(np.asarray(pil_img), cv2.COLOR_RGB2BGR)\n    if option.get('replace'):\n        img[:] = cv2_img[:]\n        return None\n    return cv2_img"

In [36]:
# coding=utf-8 show detect & recognize text
'''import cv2
import numpy as np
import matplotlib.pyplot as plt
image = cv2.imread('/content/photo_13@13-01-2022_22-50-55.jpg')
reader = easyocr.Reader(['vi'])
res = reader.readtext(image) 
for (bbox, text, prob) in res: 
  # unpack the bounding box
  (tl, tr, br, bl) = bbox
  tl = (int(tl[0]), int(tl[1]))
  tr = (int(tr[0]), int(tr[1]))
  br = (int(br[0]), int(br[1]))
  bl = (int(bl[0]), int(bl[1]))
  cv2.rectangle(image, tl, br, (0, 255, 0), 2)
  #IMG_SHAPE=(200, 520, 3)
  #np_img = np.ones(IMG_SHAPE, dtype=np.uint8) * 255
  #np_img = cv2_img_add_text(np_img, text, (tl[0],tl[1] - 10), text_rgb_color=(255, 0, 0), text_size=2)
  cv2.putText(image, text, (tl[0], tl[1] - 10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
plt.rcParams['figure.figsize'] = (16,16)
plt.imshow(image)'''

"import cv2\nimport numpy as np\nimport matplotlib.pyplot as plt\nimage = cv2.imread('/content/photo_13@13-01-2022_22-50-55.jpg')\nreader = easyocr.Reader(['vi'])\nres = reader.readtext(image) \nfor (bbox, text, prob) in res: \n  # unpack the bounding box\n  (tl, tr, br, bl) = bbox\n  tl = (int(tl[0]), int(tl[1]))\n  tr = (int(tr[0]), int(tr[1]))\n  br = (int(br[0]), int(br[1]))\n  bl = (int(bl[0]), int(bl[1]))\n  cv2.rectangle(image, tl, br, (0, 255, 0), 2)\n  #IMG_SHAPE=(200, 520, 3)\n  #np_img = np.ones(IMG_SHAPE, dtype=np.uint8) * 255\n  #np_img = cv2_img_add_text(np_img, text, (tl[0],tl[1] - 10), text_rgb_color=(255, 0, 0), text_size=2)\n  cv2.putText(image, text, (tl[0], tl[1] - 10),\n    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)\nplt.rcParams['figure.figsize'] = (16,16)\nplt.imshow(image)"

##PREDICT

In [37]:
predict_path = easyocr_non_bgr_pre_dir
data_path = easyocr_non_bgr_dir

In [38]:
easyocr_non_bgr_dir

'/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/EASYOCR/non_background/'

In [39]:
lst_file =[]
for filename in os.listdir(data_path):
  lst_file.append(filename)
try:
  lst_file.remove("Cache.cach")
  lst_file.remove("Label.txt")
  lst_file.remove("fileState.txt")
  lst_file.remove(".ipynb_checkpoints")
except:
  pass

In [40]:
#lst_file

In [41]:
len(lst_file)

90

In [42]:
#lst_file.pop(-1)

In [43]:
def predict(data_path, predict_path, lst_file):
  label_file = ""
  reader = easyocr.Reader(['vi'])
  #for i in range(len(lst_file)):
  for i in tqdm (range(len(lst_file)), desc="Loading…", ascii=False, ncols=100):
    result = reader.readtext(data_path + lst_file[i])

    label = SingleImageLabel(result, lst_file[i])
    #print("label  ", label)

    #updatelabel
    #print("UpdateLabel ",i,">>>>",lst_file[i])
    label_file = label_file + "non_background" + str("/")+str(lst_file[i])+str("\t")+str(label)+str("\n")

  SaveLabel(predict_path+"Label.txt", label_file)
  print("Done")
  return label_file

In [44]:
label_file = predict(data_path,predict_path, lst_file)

Loading…: 100%|█████████████████████████████████████████████████████| 90/90 [01:57<00:00,  1.31s/it]

Done


In [45]:
#SaveLabel("/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/EASYOCR/Label.txt", label_file)

In [46]:
SaveLabel("/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/EASYOCR/Label.txt", label_file)

##EVALUATE

In [47]:
#predict_label_path = "/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/EASYOCR/Label.txt"
groundtruth_label_path = "/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Data/Train_test_val/Label/Test_Label.txt"

In [48]:
threshold = 0.5
reader = easyocr.Reader(['vi'])

In [49]:
def convertDfTrue(groundtruth_label_path):
  f = open(groundtruth_label_path, "r")
  lst = []
  for x in f:
    temp = x.split("\t")
    lst.append(temp)

  df = pd.DataFrame(lst) 
  df.columns=["name", "detect"]
  df["points"] = np.nan
  df["text"]= np.nan
  
  final_lst =[]
  for i in tqdm (range(len(df)), desc="Loading…", ascii=False, ncols=75):
    points = []
    text= []
    temp = df["detect"].iloc[i].split('"difficult": false}')
    temp.pop(-1)
    for j in range(len(temp)):
      temp1 = temp[j].strip("[, ")+"}"
      try:
        dct_temp = ast.literal_eval(temp1)
        points.append(dct_temp["points"])
        text.append(dct_temp["transcription"])
      except: 
        temp2 = temp1.split(', "difficult": true}, ')
        temp2[0]=temp2[0]+"}"
        for h in temp2:
          dct_temp = ast.literal_eval(h)
          points.append(dct_temp["points"])
          text.append(dct_temp["transcription"])
    
    final_lst.append([[df["name"].iloc[i]],points ,text ])
  _df = pd.DataFrame(final_lst)
  #_df = _df.set_index(_df[0])
  #_df = _df.drop([0], axis=1)
  _df.columns = ["name","points", "text"]
  for i in range(len(_df)):
    _df["name"].iloc[i] = _df["name"].iloc[i][0]
    _df["name"].iloc[i] = _df["name"].iloc[i].replace("Test/","")
  return _df

In [50]:
def convertDfPre(str_):
  lst = []
  str_ = str_.split("\n")
  str_.pop(-1)
  for i in str_: 
    temp = i.split("\t")
    lst.append(temp)

  df = pd.DataFrame(lst) 
  df.columns=["name", "detect"]
  df["points"] = np.nan
  df["text"]= np.nan
  final_lst =[]
  for i in tqdm (range(len(df)), desc="Loading…", ascii=False, ncols=75):
    #try:
    points = []
    text= []
    temp = df["detect"].iloc[i].split(", 'difficult': False},")
    #temp.pop(-1)
    try:
      for j in range(len(temp)):
        temp1 = temp[j].strip("[, ")+"}"
        try:
          dct_temp = ast.literal_eval(temp1)
          points.append(dct_temp["points"])
          text.append(dct_temp["transcription"])
        except: 
          temp2 = temp1.replace(", 'difficult': False}]}", "")
          temp2 = temp2+"}"
          dct_temp = ast.literal_eval(temp2)
          points.append(dct_temp["points"])
          text.append(dct_temp["transcription"])
    except: 
      pass
    final_lst.append([[df["name"].iloc[i]],points ,text ])
  _df = pd.DataFrame(final_lst)
  print(_df.columns)
  #_df = _df.set_index(_df[0])
  #_df = _df.drop([0], axis=1)
  _df.columns = ["name","points", "text"]
  
  for i in range(len(_df)):
    _df["name"].iloc[i] = _df["name"].iloc[i][0]
    _df["name"].iloc[i] = _df["name"].iloc[i].replace("non_background/","")
    #_df["name"].iloc[i] = _df["name"].iloc[i].replace("non_backgr","")
  return _df


In [51]:
print("predicted list")
final_pre_df = convertDfPre(label_file)
print("groundtrurh list")
final_true_df = convertDfTrue(groundtruth_label_path)

predicted list


Loading…: 100%|██████████████████████████| 90/90 [00:00<00:00, 1163.92it/s]


RangeIndex(start=0, stop=3, step=1)
groundtrurh list


Loading…: 100%|██████████████████████████| 90/90 [00:00<00:00, 1982.59it/s]


In [52]:
final_pre_df.iloc[0]

name                       photo_43@05-01-2022_08-41-24.jpg
points    [[[386, 274], [532, 274], [532, 298], [386, 29...
text      [BO GIÁO DUC VA, Cuumg   tUPT_rg' PInh hhiér, ...
Name: 0, dtype: object

In [53]:
final_true_df.iloc[0]

name                        photo_1@05-01-2022_08-40-54.jpg
points    [[[487, 176], [749, 212], [746, 234], [484, 19...
text      [BỘ GIÁO DỤC VÀ ĐÀO TẠO, CÔNG NGHỆ, NÔNG, LÂM,...
Name: 0, dtype: object

In [54]:
#result = reader.readtext(saving_path+"photo_2@13-01-2022_22-50-51non_backgr.jpg")

In [55]:
def CalculateIou(lst1, lst2):
  polygon1 = Polygon(np.array(lst1))
  polygon2 = Polygon(np.array(lst2))
  intersect = polygon1.intersection(polygon2).area
  union = polygon1.union(polygon2).area
  iou = intersect / union
  #print(iou)
  return iou

In [56]:
def CalculateScore_image(name,threshold ): 
  true = final_true_df[final_true_df["name"] == name]["points"].values
  pre  = final_pre_df[final_pre_df["name"] == name]["points"].values

  temp_lst = []
  for i in range(len(pre[0])):
    for j in range(len(true[0])):
      iou = CalculateIou(pre[0][i], true[0][j])
      temp_lst.append([i, j, iou])
  temp_df = pd.DataFrame(temp_lst, columns = ["pol_pre", "pol_true", "iou"])
  temp_df = temp_df.sort_values(["iou", "pol_true"], ascending=False)
  temp_df = temp_df.reset_index()

  overlap = {}
  temp_df["select"]=np.nan
  temp_df["evaluate"]  = np.nan
  for i in range(len(pre[0]) *len(true[0])):
    if temp_df["pol_true"].iloc[i] not in overlap.keys():
      overlap.update({temp_df["pol_true"].iloc[i]:temp_df["pol_pre"].iloc[i]})
      temp_df["select"].iloc[i] =1
    
  for i in range(len(pre[0]) *len(true[0])):
    if (temp_df["iou"].iloc[i] >= threshold and temp_df["select"].iloc[i] == 1.0):
      temp_df["evaluate"].iloc[i] = "True Positive"
    if (temp_df["iou"].iloc[i] >= threshold and temp_df["select"].iloc[i] != 1.0):
      temp_df["evaluate"].iloc[i] = "False Positive"
    if (temp_df["iou"].iloc[i] <= threshold and temp_df["select"].iloc[i] == 1.0):
      temp_df["evaluate"].iloc[i] = "False Negative"
  #temp_df = temp_df[temp_df["select"] == 1]

  #iou = list(temp_df["iou"])
  #REDUNDANT
  redundant = []
  temp_re = set(list(temp_df["pol_pre"]))
  for i in temp_re: 
    if i not in list(temp_df[temp_df["select"]==1]["pol_pre"]):
      #iou.append(0)
      redundant.append("FP")

  #MISS
  miss = []
  temp_miss = set(list(temp_df["pol_true"]))
  for i in temp_miss: 
    if i not in list(temp_df[temp_df["select"]==1]["pol_true"]):
      #iou.append(0)
      miss.append("FN")

  FN = len(miss) + len(temp_df[temp_df["evaluate"]=="False Negative"])
  TP = len(temp_df[temp_df["evaluate"]=="True Positive"]) 
  FP = len(temp_df[temp_df["evaluate"]=="False Positive"]) +len(redundant)
  precision = TP/(TP + FP)
  recall    = TP/(TP + FN)
  final_df = temp_df[temp_df["evaluate"]== "True Positive"]
  final_df.to_csv("/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/EASYOCR/points_text_non_background/"+name+".csv", index = None)
  return precision, recall

In [57]:
#CalculateScore_image(name, threshold)

In [58]:
def CalculateIou_Test(lst_file, thresold):
  precision = []
  recall = []
  for i in tqdm (range(len(lst_file)), desc="Loading…", ascii=False, ncols=75): 
  #for i in range(len(lst_file)):
    #print(i)
    try:
      p, r = CalculateScore_image(lst_file[i], threshold)
      if(precision !=0 and recall != 0):
        precision.append(p)
        recall.append(r)
    except: 
      pass
  pre = mean(precision)
  re  = mean(recall)
  return pre, re

In [59]:
precision50,recall50 = CalculateIou_Test(lst_file, threshold)

Loading…:   0%|                                     | 0/90 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
Loading…: 100%|████████████████████████████| 90/90 [00:06<00:00, 13.08it/s]


In [60]:
print("Precision: ",precision50,"\n", "Recall: ", recall50)

Precision:  0.5017561246260286 
 Recall:  0.6108490448378089


##save list_label

In [61]:
#label_file = "\n".join(label_file)

In [62]:
f = open(predict_path+"label_file.txt", "w")
f.write(label_file)
f.close()

##Score Recognition

In [63]:
final_pre_df = convertDfPre(label_file)

Loading…: 100%|██████████████████████████| 90/90 [00:00<00:00, 1098.87it/s]

RangeIndex(start=0, stop=3, step=1)


In [64]:
final_true_df = convertDfTrue(groundtruth_label_path)

Loading…: 100%|██████████████████████████| 90/90 [00:00<00:00, 1572.84it/s]


In [65]:
point_text = "/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/EASYOCR/points_text_non_background/"

In [66]:
def Calculate_score_text(final_true_df, final_pre_df):
  #cer_lst = []
  #wer_lst = []
  true = []
  pre = []
  for i in range(len(lst_file)):
    name = lst_file[i]
    try:
      df = pd.read_csv(point_text+name+".csv")
      true_lst = []
      pre_lst  = []
      for i in range(len(df)):
        true_lst.append(final_true_df[final_true_df["name"]==name]["text"].values[0][df["pol_true"].iloc[i]])
        pre_lst.append(final_pre_df[final_pre_df["name"]==name]["text"].values[0][df["pol_pre"].iloc[i]])
      true.append(true_lst)
      pre.append(pre_lst)
    except:
      pass
    #w = wer(true_lst, pre_lst)
    #c = cer(true_lst, pre_lst)
    #cer_lst.append(c)
    #wer_lst.append(w)
  #_cer = mean(cer_lst)
  #_wer = mean(wer_lst)
  return true, pre


In [67]:
true = []
pre = []
true,pre =  Calculate_score_text(final_true_df, final_pre_df)

In [68]:
import numpy as np
#!pip install sklearn
#import scipy
#import sklearn
from sklearn.metrics import precision_recall_fscore_support
y_true = np.array(true[0])
y_pred = np.array(pre[0])

In [69]:
true[2][2].split(" ")

['CÓ', 'ĐÁP', 'ÁN']

In [70]:
pre[2][2].split(" ")

['56ĐÁPÁN']

In [71]:
def get_lst_true_pre_each_image(true, pre):
  lst_true = []
  lst_pre = []
  for i in range(len(true)):
    #print(i)
    temp_true = true[i].split(" ")
    temp_pre = pre[i].split(" ")
    
    for j in range(len(temp_true)):
      lst_true.append(temp_true[j])
    for j in range(len(temp_pre)):
      lst_pre.append(temp_pre[j])
    hon = len(lst_true) - len(lst_pre)
    if hon > 0: 
      for j in range(hon):
        lst_pre.append("")
    else:
      for j in range(hon*(-1)):
        lst_true.append("")
  return lst_true, lst_pre

In [72]:
t, p = get_lst_true_pre_each_image(true[2], pre[2])
print(t)
print(p)

['11', 'ANH', 'CÓ', 'ĐÁP', 'ÁN', 'TIẾNG', 'BÀI', 'TẬP', 'NGUYỄN', 'THANH', 'LOAN', 'NHÀ', 'XUẤT', 'BẢN', 'ĐÀ', 'NẴNG', 'MAI', 'LAN', 'HƯƠNG']
['1l', 'ANH', '56ĐÁPÁN', '', '', 'TIẼìxề', 'BAI', 'TÂP', 'NGUYẾN', 'THANH', 'LOAN', 'NHÀ', 'XUẤT', 'BẢN', 'ĐÀ', 'NẴNC', 'MAI', 'LAN', 'HUONG']


In [73]:
def cal_test_score(true, pre):
  lst_score = []
  for i in range(len(true)):
    lst_true, lst_pre = get_lst_true_pre_each_image(true[i], pre[i])
    score = list(precision_recall_fscore_support(lst_true, lst_pre, average='micro'))
    #print(i,">>>",score)
    lst_score.append(score[2])
  f1 = mean(lst_score)
  return f1

In [74]:
h_mean  = cal_test_score(true, pre)
print("H-mean score ", h_mean)

H-mean score  0.45933341712451004


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarn